In [1]:
import json
from enum import Enum
from gurobipy import Model, GRB, quicksum
import numpy as np
import pandas as pd
from random import randint


In [2]:
class DATASET(Enum):
    TOY = "toy"
    MEDIUM = "medium"
    LARGE = "large"

In [3]:
def load_data(name):
    """name must be an instance of DATASET like DATASET.TOY for example"""
    if not isinstance(name, DATASET):
        raise TypeError("direction must be an instance of DATASET Enum")
    with open(f"../data/{name.value}_instance.json", "r") as f:
        data = json.load(f)
    return data


def get_dims(data):
    return (
        len(data["staff"]),
        data["horizon"],
        len(data["qualifications"]),
        len(data["jobs"]),
    )

def get_qualification_index(list_qualifications, qualification): # qualification is "A", "B", "C" ...
    return list_qualifications.index(qualification)

In [4]:
def init_model():
    m = Model("Project modelling")
    return m

In [5]:
def create_decision_variables(model, n_staff, horizon, n_qualifs, n_jobs):
    X = model.addVars(n_staff, horizon, n_qualifs, n_jobs, vtype=GRB.BINARY, name="assignements")
    J = model.addVars(n_jobs, vtype=GRB.BINARY, name="completion")
    D = model.addVars(n_jobs, lb=0, ub=horizon + 1, vtype=GRB.INTEGER, name="end_dates")
    L = model.addVars(n_jobs, lb=0, ub=horizon + 1, vtype=GRB.INTEGER, name="nb_dates_late")
    return model, X, J, D, L


In [6]:
def add_constraints_end_dates(model, X, D, n_staff, horizon, n_qualifs, n_jobs):
    model.addConstrs(
        X[i, j, k, l] * j <= D[l] 
        for i in range(n_staff)
        for j in range(horizon)
        for k in range(n_qualifs)
        for l in range(n_jobs)
    )

In [7]:
def add_constraints_lateness(model, D, L, jobs, n_staff, horizon, n_qualifs, n_jobs):
    model.addConstrs(
        D[l] - jobs[l]["due_date"] <= L[l] for l in range(n_jobs)
    )

In [8]:
def add_constraints_worked_days_below_required_days(model, X, jobs, qualifications, n_staff, horizon, n_jobs):
    model.addConstrs(quicksum(X[i,j,get_qualification_index(qualifications, k),l] for i in range(n_staff) for j in range(horizon)) <= jobs[l]["working_days_per_qualification"][k] for l in range(n_jobs) for k in jobs[l]["working_days_per_qualification"].keys())
    

In [9]:
def add_constraints_worked_days_above_required_days(model, X, J, jobs, qualifications, n_staff, horizon, n_jobs):
    model.addConstrs(quicksum(X[i,j,get_qualification_index(qualifications, k),l] for i in range(n_staff) for j in range(horizon)) >= J[l]* jobs[l]["working_days_per_qualification"][k] for l in range(n_jobs) for k in jobs[l]["working_days_per_qualification"].keys())

    # for l in range(n_jobs): 
    #     for quali,njk in data["jobs"][l]["working_days_per_qualification"].items():
    #         model.addConstr(J[l]*njk <= quicksum([X[i,j,get_qualification_index(qualifications, quali),l] for i in range(n_staff) for j in range(horizon)]))

In [10]:
def in_qualification(data, i, k):
    return data["qualifications"][k] in data["staff"][i]["qualifications"]


def add_qualification_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data):
    model.addConstrs(
        X[i, j, k, l] == 0
        for i in range(n_staff)
        for j in range(horizon)
        for k in range(n_qualifs)
        for l in range(n_jobs)
        if not in_qualification(data, i, k)
    )


def in_vacation(i, j, data):
    data = {l: data["staff"][l] for l in range(len(data["staff"]))}
    data = data[i]["vacations"]
    return j in data


def add_vacation_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data):
    model.addConstrs(
        X[i, j, k, l] == 0
        for i in range(n_staff)
        for j in range(horizon)
        for k in range(n_qualifs)
        for l in range(n_jobs)
        if in_vacation(i, j, data)
    )

In [11]:
def add_constraint_employees_working_only_one_day(model, X, J, data, n_staff, n_jobs, horizon, n_qualifs):
    model.addConstrs(quicksum(X[i,j,k,l] for l in range(n_jobs) for k in range (n_qualifs)) <= 1 for i in range(n_staff) for j in range(horizon))

In [12]:
def add_profit_as_first_objective(model, J, L, jobs):
    model.setObjective(
        quicksum( (J[index_job] * job["gain"] - job["daily_penalty"] * L[index_job]) for index_job, job in enumerate(jobs)),
        GRB.MAXIMIZE
    )
    return model

In [13]:

# Importing data
data = load_data(DATASET.TOY)
n_staff, horizon, n_qualifs, n_jobs = get_dims(data)

# Instanciation du modèle
model = init_model()

# Création des variables : binaires dans X et J, entières de 0 à horizon + 3
model, X, J, D, L = create_decision_variables(model, n_staff, horizon, n_qualifs, n_jobs)

# maj du modèle
model.update()

print(in_qualification(data, 1, 1))

# Ajout des constraintes

add_constraint_employees_working_only_one_day(model, X ,J,data,n_staff,n_jobs,horizon, n_qualifs)
add_qualification_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data)
add_vacation_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data)

add_constraints_end_dates(model, X, D, n_staff, horizon, n_qualifs, n_jobs)
add_constraints_lateness(model, D, L, data["jobs"], n_staff, horizon, n_qualifs, n_jobs)

add_constraints_worked_days_below_required_days(model, X, data["jobs"], data["qualifications"], n_staff, horizon, n_jobs)
add_constraints_worked_days_above_required_days(model, X , J, data["jobs"], data["qualifications"], n_staff, horizon, n_jobs)

# maj du modèle
model.update()

# Fonction Objectif
model = add_profit_as_first_objective(model, J, L, data["jobs"])

# maj du modèle
model.update()

# Paramétrage (mode mute)
# model.params.outputflag = 0

# Résolution du PL
model.optimize()

staffos = {i:[] for i in range(n_staff)}
for i in range(n_staff):
    for j in range(horizon):
        done = False
        for k in range(n_qualifs):
            for l in range(n_jobs):
                if X[(i, j, k, l)].X==1:
                    staffos[i].append([k,l])
                    done = True
        if done==False:
            staffos[i].append([-1,-1])
            
df=pd.DataFrame(staffos).T
df.index=[data["staff"][i]["name"] for i in range(n_staff)]
def format_df(styler):
    color = []
    for i in range(n_jobs):
        color.append('#%06X' % randint(0, 0xFFFFFF))
    styler.applymap(lambda x: f"background-color: {color[x[1]]}" if x[0]!=-1 else "background-color: black")
    styler.format(lambda x: data["qualifications"][x[0]] if x[0]!=-1 else "")
    return styler
    
df.style.pipe(format_df)



Restricted license - for non-production use only - expires 2024-10-28
True
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 370 rows, 240 columns and 1055 nonzeros
Model fingerprint: 0xcb56a9c0
Variable types: 0 continuous, 240 integer (230 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [3e+00, 2e+01]
  Bounds range     [1e+00, 6e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 329 rows and 142 columns
Presolve time: 0.00s
Presolved: 41 rows, 98 columns, 304 nonzeros
Variable types: 0 continuous, 98 integer (98 binary)
Found heuristic solution: objective 20.0000000

Root relaxation: objective 7.500000e+01, 57 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |    

,0,1,2,3,4
Olivia,B,A,B,B,C
Liam,B,,A,B,A
Emma,C,C,,C,C


In [14]:
# # Objective Function 1: Maximize Profits
# model.setObjective(sum(profit[j]*y[j] for j in projects) - sum(penalty[i,j]*x[i,j,k] for i in staff for j in qualifications for k in working_days), GRB.MAXIMIZE)

# # Objective Function 2: Minimize Number of Projects per Employee
# model.setObjective(sum(x[i,j,k] for i in staff for j in qualifications for k in working_days) , GRB.MINIMIZE)
# model.setObjective(sum(x[i,j,k] for i in staff for j in qualifications for k in working_days) , GRB.MINIMIZE)

# # Objective Function 3: Minimize Length of Longest Project
# model.setObjective(max(duration[j] for j in projects), GRB.MINIMIZE)